In [1]:
import sqlite3
def run_query(query):
    conn = sqlite3.connect("sqlite_db_pythonsqlite.db")
    cur = conn.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    for row in rows:
        print(row)

query = """
        SELECT *
        FROM FACILITIES
        """
run_query(query)

(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [2]:
# Q1: Some of the facilities charge a fee to members, but some do not.
# Write a SQL query to produce a list of the names of the facilities that do.

query = """
        SELECT name 
        FROM Facilities
        WHERE membercost > 0;
        """
run_query(query)

('Tennis Court 1',)
('Tennis Court 2',)
('Massage Room 1',)
('Massage Room 2',)
('Squash Court',)


In [3]:
# Q2: How many facilities do not charge a fee to members?

query = """
        SELECT COUNT(*)
        FROM Facilities
        WHERE membercost = 0;
        """

run_query(query)

(4,)


In [4]:
# Q3: Write an SQL query to show a list of facilities that charge a fee to members,
# where the fee is less than 20% of the facility's monthly maintenance cost.
# Return the facid, facility name, member cost, and monthly maintenance of the
# facilities in question.

query = """
        SELECT facid, name, membercost, monthlymaintenance
        FROM Facilities
        WHERE membercost < (0.2 * monthlymaintenance)
            AND membercost > 0
        GROUP BY facid;
        """

run_query(query)

(0, 'Tennis Court 1', 5, 200)
(1, 'Tennis Court 2', 5, 200)
(4, 'Massage Room 1', 9.9, 3000)
(5, 'Massage Room 2', 9.9, 3000)
(6, 'Squash Court', 3.5, 80)


In [5]:
# Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
# Try writing the query without using the OR operator.

query = """
        SELECT *
        FROM Facilities
        WHERE facid IN (1,5);
        """

run_query(query)

(1, 'Tennis Court 2', 5, 25, 8000, 200)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)


In [6]:
# Q5: Produce a list of facilities, with each labelled as
# 'cheap' or 'expensive', depending on if their monthly maintenance cost is
# more than $100. Return the name and monthly maintenance of the facilities
# in question.

query = """
        SELECT  facid, 
                name,
                CASE WHEN monthlymaintenance > 100 THEN 'expensive'
                     ELSE 'cheap' END AS price_level
        FROM Facilities;
        """

run_query(query)

(0, 'Tennis Court 1', 'expensive')
(1, 'Tennis Court 2', 'expensive')
(2, 'Badminton Court', 'cheap')
(3, 'Table Tennis', 'cheap')
(4, 'Massage Room 1', 'expensive')
(5, 'Massage Room 2', 'expensive')
(6, 'Squash Court', 'cheap')
(7, 'Snooker Table', 'cheap')
(8, 'Pool Table', 'cheap')


In [7]:
# Q6: You'd like to get the first and last name of the last member(s)
# who signed up. Try not to use the LIMIT clause for your solution. */


query = """
        SELECT firstname, surname, joindate
        FROM Members
        WHERE surname <> "GUEST"
        ORDER BY joindate DESC
        LIMIT 1;
        """

run_query(query)

('Darren', 'Smith', '2012-09-26 18:08:45')


In [8]:
# Q7: Produce a list of all members who have used a tennis court.
# Include in your output the name of the court, and the name of the member
# formatted as a single column. Ensure no duplicate data, and order by
# the member name.

query = """
        SELECT DISTINCT(memid), name, (firstname || ' ' || surname) AS member
        FROM Bookings
        LEFT JOIN Facilities
        USING (facid)
        LEFT JOIN Members
        USING (memid)
        WHERE facid IN (0,1)
            AND surname <> "GUEST"
        ORDER BY member;
        """

run_query(query)

(12, 'Tennis Court 1', 'Anne Baker')
(12, 'Tennis Court 2', 'Anne Baker')
(6, 'Tennis Court 2', 'Burton Tracy')
(6, 'Tennis Court 1', 'Burton Tracy')
(10, 'Tennis Court 1', 'Charles Owen')
(10, 'Tennis Court 2', 'Charles Owen')
(1, 'Tennis Court 2', 'Darren Smith')
(28, 'Tennis Court 1', 'David Farrell')
(28, 'Tennis Court 2', 'David Farrell')
(11, 'Tennis Court 2', 'David Jones')
(11, 'Tennis Court 1', 'David Jones')
(17, 'Tennis Court 1', 'David Pinker')
(26, 'Tennis Court 1', 'Douglas Jones')
(36, 'Tennis Court 1', 'Erica Crumpet')
(15, 'Tennis Court 2', 'Florence Bader')
(15, 'Tennis Court 1', 'Florence Bader')
(5, 'Tennis Court 1', 'Gerald Butters')
(5, 'Tennis Court 2', 'Gerald Butters')
(27, 'Tennis Court 2', 'Henrietta Rumney')
(14, 'Tennis Court 1', 'Jack Smith')
(14, 'Tennis Court 2', 'Jack Smith')
(4, 'Tennis Court 1', 'Janice Joplette')
(4, 'Tennis Court 2', 'Janice Joplette')
(13, 'Tennis Court 2', 'Jemima Farrell')
(13, 'Tennis Court 1', 'Jemima Farrell')
(22, 'Tennis Cou

In [9]:
# Q8: Produce a list of bookings on the day of 2012-09-14 which
# will cost the member (or guest) more than $30. Remember that guests have
# different costs to members (the listed costs are per half-hour 'slot'), and
# the guest user's ID is always 0. Include in your output the name of the
# facility, the name of the member formatted as a single column, and the cost.
# Order by descending cost, and do not use any subqueries.

query = """
        SELECT  name, 
                (firstname || ' ' || surname) AS member,
                (CASE WHEN memid = 0 THEN (slots * guestcost)
                    ELSE (slots * membercost) END) AS cost
        FROM Bookings
        LEFT JOIN Facilities
        USING(facid)
        LEFT JOIN Members
        USING(memid)
        WHERE starttime LIKE '2012-09-14%'
            AND (CASE WHEN memid = 0 THEN guestcost
                    ELSE membercost END) > 30
        ORDER BY cost DESC;
        """

run_query(query)

('Massage Room 2', 'GUEST GUEST', 320)
('Massage Room 1', 'GUEST GUEST', 160)
('Massage Room 1', 'GUEST GUEST', 160)
('Massage Room 1', 'GUEST GUEST', 160)


In [10]:
# Q9: This time, produce the same result as in Q8, but using a subquery.

query = """
        SELECT name, member, cost
        FROM (SELECT name,
                     (firstname || ' ' || surname) AS member,
                     (CASE WHEN memid = 0 THEN guestcost
                        ELSE membercost END) AS cost
                FROM Bookings
                LEFT JOIN Facilities
                USING (facid)
                LEFT JOIN Members
                USING (memid)
                WHERE starttime LIKE '2012-09-14%') AS sub
        WHERE cost > 30 
        ORDER BY cost DESC;
        """

run_query(query)

('Massage Room 1', 'GUEST GUEST', 80)
('Massage Room 1', 'GUEST GUEST', 80)
('Massage Room 1', 'GUEST GUEST', 80)
('Massage Room 2', 'GUEST GUEST', 80)


In [11]:
# Q10: Produce a list of facilities with a total revenue less than 1000.
# The output of facility name and total revenue, sorted by revenue. Remember
# that there's a different cost for guests and members! 

query = """
        SELECT name, total_rev
        FROM (SELECT name, SUM(revenue) AS total_rev
            FROM (SELECT  name,
                    CASE WHEN memid = 0 THEN (slots * guestcost)
                        ELSE (slots * membercost) END AS revenue
                    FROM Bookings
                    LEFT JOIN Facilities
                    USING(facid)
                    LEFT JOIN Members
                    USING(memid)) AS sub1
            GROUP BY name) AS sub2
        WHERE total_rev < 1000
        ORDER BY total_rev;
        """

run_query(query)

('Table Tennis', 180)
('Snooker Table', 240)
('Pool Table', 270)


In [12]:
# Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

query = """
        SELECT (m.firstname || ' ' || m.surname) AS member, r.firstname AS recommender_first, r.surname AS recommender_last
        FROM Members AS m
        INNER JOIN Members as r
        ON m.memid = r.recommendedby
        ORDER BY recommender_last, recommender_first
        """

run_query(query)


('Ponder Stibbons', 'Florence', 'Bader')
('Ponder Stibbons', 'Anne', 'Baker')
('Jemima Farrell', 'Timothy', 'Baker')
('Tim Rownam', 'Tim', 'Boothe')
('Darren Smith', 'Gerald', 'Butters')
('Timothy Baker', 'Joan', 'Coplin')
('Tracy Smith', 'Erica', 'Crumpet')
('Janice Joplette', 'Nancy', 'Dare')
('Gerald Butters', 'Matthew', 'Genting')
('Millicent Purview', 'John', 'Hunt')
('Janice Joplette', 'David', 'Jones')
('David Jones', 'Douglas', 'Jones')
('Darren Smith', 'Janice', 'Joplette')
('Darren Smith', 'Anna', 'Mackenzie')
('Darren Smith', 'Charles', 'Owen')
('Jemima Farrell', 'David', 'Pinker')
('Tracy Smith', 'Millicent', 'Purview')
('Matthew Genting', 'Henrietta', 'Rumney')
('Florence Bader', 'Ramnaresh', 'Sarwin')
('Darren Smith', 'Jack', 'Smith')
('Burton Tracy', 'Ponder', 'Stibbons')
('Tracy Smith', 'Henry', 'Worthington-Smyth')


In [13]:
# Q12: Find the facilities with their usage by member, but not guests 

query = """
        SELECT name, SUM(slots)
        FROM Bookings
        LEFT JOIN Facilities
        USING (facid)
        WHERE memid > 0
        GROUP BY facid
        """

run_query(query)

('Tennis Court 1', 957)
('Tennis Court 2', 882)
('Badminton Court', 1086)
('Table Tennis', 794)
('Massage Room 1', 884)
('Massage Room 2', 54)
('Squash Court', 418)
('Snooker Table', 860)
('Pool Table', 856)


In [14]:
# Q13: Find the facilities usage by month, but not guests 

query = """
        SELECT name, SUM(slots) AS uses, strftime('%m', starttime) AS month
        FROM Bookings
        LEFT JOIN Facilities
        USING (facid)
        WHERE memid > 0
        GROUP BY month, facid
        ORDER BY month, name
        """

run_query(query)

('Badminton Court', 165, '07')
('Massage Room 1', 166, '07')
('Massage Room 2', 8, '07')
('Pool Table', 110, '07')
('Snooker Table', 140, '07')
('Squash Court', 50, '07')
('Table Tennis', 98, '07')
('Tennis Court 1', 201, '07')
('Tennis Court 2', 123, '07')
('Badminton Court', 414, '08')
('Massage Room 1', 316, '08')
('Massage Room 2', 18, '08')
('Pool Table', 303, '08')
('Snooker Table', 316, '08')
('Squash Court', 184, '08')
('Table Tennis', 296, '08')
('Tennis Court 1', 339, '08')
('Tennis Court 2', 345, '08')
('Badminton Court', 507, '09')
('Massage Room 1', 402, '09')
('Massage Room 2', 28, '09')
('Pool Table', 443, '09')
('Snooker Table', 404, '09')
('Squash Court', 184, '09')
('Table Tennis', 400, '09')
('Tennis Court 1', 417, '09')
('Tennis Court 2', 414, '09')
